In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
!wget https://github.com/hse-ds/iad-intro-ds/raw/master/2023/homeworks/Data.zip
!unzip /content/Data.zip

--2024-02-26 13:32:10--  https://github.com/hse-ds/iad-intro-ds/raw/master/2023/homeworks/Data.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hse-ds/iad-intro-ds/master/2023/homeworks/Data.zip [following]
--2024-02-26 13:32:10--  https://raw.githubusercontent.com/hse-ds/iad-intro-ds/master/2023/homeworks/Data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16727 (16K) [application/zip]
Saving to: ‘Data.zip.1’

Data.zip.1          100%[===================>]  16.33K  --.-KB/s    in 0s      

2024-02-26 13:32:10 (70.7 MB/s) - ‘Data.zip.1’ saved [16727/16727]

Archive:  /content/Data.zi

#### Описание данных

В папке Dat (https://github.com/hse-ds/iad-intro-ds/blob/master/2023/homeworks/Data.zip) находится информация о студентах. Всего 10 групп студентов. Файлы делятся на две категории:
    * Students_info_i - информация о студентах из группы i
    * Students_marks_i - оценки студентов из группы i за экзамены

### Одно из важных достоинств pandas $-$ это удобные методы реляционного взаимодействия с данными, аналогичные, например, возможностям SQL для слияния и конкатенации таблиц: merge, join, concat. Наличие готовых методов позволяет не реализовывать самостоятельно поэлементную обработку данных и оперировать сразу целыми таблицами данных.

Подробно об этих методах: https://www.kaggle.com/residentmario/renaming-and-combining#Combining

#### Соберем всю информацию о студентах в одну таблицу df. В получившейся таблице должна быть информация и оценки всех студентов из всех групп. Напечатаем несколько строк таблицы для демонстрации результата.¶

In [ ]:
df = pd.DataFrame()
for i in range(10):
  current_info = pd.read_csv("Data/Students_info_" + str(i) + ".csv")
  current_marks = pd.read_csv("Data/Students_marks_" + str(i) + ".csv")

  # Объединяем таблицу с информацией о студентах группы i и
  # таблицу с оценками студентов группы i (не добавляем столбец index повторно)
  current = pd.concat([current_info,
                       current_marks[current_marks.columns[1:]]], axis=1)

  # Добавляем информацию о текущей группе в общую таблицу
  df = pd.concat([df, current])

# Выполним вывод, который покажет все столбцы, первые пять и
# последние пять строк, чтобы убедиться в правильности построения.
df

,index,gender,race/ethnicity,parental level of education,lunch,test preparation course,group,math score,reading score,writing score
0,0,female,group B,bachelor's degree,standard,none,group1,72,72,74
1,1,female,group C,some college,standard,completed,group1,69,90,88
2,2,female,group B,master's degree,standard,none,group1,90,95,93
3,3,male,group A,associate's degree,free/reduced,none,group1,47,57,44
4,4,male,group C,some college,standard,none,group1,76,78,75
...,...,...,...,...,...,...,...,...,...,...
95,995,female,group E,master's degree,standard,completed,group10,88,99,95
96,996,male,group C,high school,free/reduced,none,group10,62,55,55
97,997,female,group C,high school,free/reduced,completed,group10,59,71,65
98,998,female,group D,some college,standard,completed,group10,68,78,77


#### Удалим столбец index у полученной таблицы. Напечатаем первые 10 строк таблицы.

In [ ]:
df = df[df.columns[1:]] # Удаляем первый столбец
df.head(10)

#### Выведем на экран размеры полученной таблицы

In [ ]:
df.shape
# df.shape возращает кортеж, в котором первое значение -- кол-во строк в df,
#                                      второе значение -- кол-во столбцов в df

(1000, 9)

#### Выведем на экран статистические характеристики числовых столбцов таблицы (минимум, максимум, среднее значение, стандартное отклонение)

In [ ]:
# Сначала получим требуемые характеристики по каждому предмету,
# затем объединим информацию по каждому предмету в один DataFrame
pd.concat([pd.concat([df['math score'].describe()[1:4], df['math score'].describe()[-1:]]),
           pd.concat([df['reading score'].describe()[1:4], df['reading score'].describe()[-1:]]),
           pd.concat([df['writing score'].describe()[1:4], df['writing score'].describe()[-1:]])],
          axis=1)

,math score,reading score,writing score
mean,66.08900,69.169000,68.054000
std,15.16308,14.600192,15.195657
min,0.00000,17.000000,10.000000
max,100.00000,100.000000,100.000000


#### Проверем, есть ли в таблице пропущенные значения

In [ ]:
print(df.isna().any().any())

False


df.isna() создает новый DataFrame, где каждая ячейка показывает, есть ли в исходном DataFrame пропущенное значение.

df.isna().any() проверяет каждый столбец на наличие хотя бы одного True и возвращает серию булевых значений, соответствующих столбцам.

df.isna().any().any() дает одно булево значение, указывающее, есть ли во всем DataFrame хотя бы одно пропущенное значение.

#### Выведем на экран средние баллы студентов по каждому предмету (math, reading, writing)

In [ ]:
print(pd.DataFrame({'math': [df['math score'].mean()],
              'reading': [df['reading score'].mean()],
              'writing': [df['writing score'].mean()]}, index=['mean']))

        math  reading  writing
mean  66.089   69.169   68.054


Создаем DataFrame, в котором столбцами являются предметы,
единственная строка отвечает за средний балл по каждому предмету.
Здесь можно было воспользоваться и Series, потому что строка всего одна.
Но я захотела подписать и столбцы, и строку для удобного представления
информации, поэтому создала DataFrame.

**Задание 7. Как зависят оценки от того, проходил ли студент курс для подготовки к сдаче экзамена (test preparation course)? Выведите на экран для каждого предмета в отдельности средний балл студентов, проходивших курс для подготовки к экзамену и не проходивших курс.**

In [ ]:
completed = df[df['test preparation course'] == 'completed']
none = df[df['test preparation course'] == 'none']

pd.DataFrame({'GPA with preparation course': [completed['math score'].mean(),
                                              completed['reading score'].mean(),
                                              completed['writing score'].mean()],
              'GPA without preparation course': [none['math score'].mean(),
                                                 none['reading score'].mean(),
                                                 none['writing score'].mean()]},
             index=['math', 'reading', 'writing'])

,GPA with preparation course,GPA without preparation course
math,69.695531,64.077882
reading,73.893855,66.534268
writing,74.418994,64.504673


По каждому из предметов средний балл (GPA) студентов, проходивших курс подготовки к экзамену, выше по сравнению со студентами, не проходившими такой курс. Это может указывать на то, что подготовительный курс помогает улучшить понимание материала и подготовку к экзаменам, что, в свою очередь, приводит к более высоким оценкам в экзаменационных предметах.

Стоит отметить, что самая большая разница (почти в 10 баллов) возникает между средними оценками по предмету writing, а самая маленькая разница (почти в 5,5 баллов) -- по предмету math. Также стоит учесть, что средний балл без прохождения курса по этим предметам практически одинаков. Из этого можно предположить, что на подготовительном курсе обучение предмету writing более продуктивно с точки зрения подготовки к экзамену. Или, возможно, на курс чаще записываются студенты с низким баллом (знаниями) по предмету math и высокими познаниями в предмете writing изначально (например, выпускники школ с уклонном на гуманитарные предметы), тем самым "скрашивая" итоговую статистику по второму предмету.

Это только предварительные наблюдения, и для более точных выводов потребуется более глубокий статистический анализ.

**Задание 8. Выведите на экран все различные значения из столбца lunch.**

In [ ]:
print(df['lunch'].unique())

# df['lunch'] возвращает Series со значениями из столбца lunch,
# а unique() возвращает array с уникальными значениями

['standard' 'free/reduced']


**Задание 9. Переименуйте колонку "parental level of education" в "education", а "test preparation course" в "test preparation" с помощью метода pandas rename**
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html

In [ ]:
df = df.rename(columns={'parental level of education': 'education',
                   'test preparation course': 'test preparation'})

# Метод rename() принимает словарь, где ключи - старые имена столбцов,
# а значения - новые имена

**Зафиксируем минимальный балл для сдачи экзамена**

In [ ]:
passmark = 50

**Задание 10. Ответьте на вопросы:**
    * Какая доля студентов сдала экзамен по математике (passmark >= 50)?
    * Какая доля студентов, проходивших курс подготовки к экзамену, сдала экзамен по математике?
    * Какая доля женщин, не проходивших курс подготовки к экзамену, не сдала экзамен по математике?

In [ ]:
print(df[df['math score'] >= passmark].shape[0] / df.shape[0])

print(completed[completed['math score'] >= passmark].shape[0] / completed.shape[0])

none_compl_and_female = none[none['gender'] == 'female']
print(none_compl_and_female[none_compl_and_female['math score'] < passmark].shape[0] /
      none_compl_and_female.shape[0])

0.865
0.9217877094972067
0.20958083832335328


Доля студентов, сдавших экзамен по математике: 0.865.

Доля студентов, проходивших курс подготовки к экзамену, которые сдали экзамен по математике: 0.922.

Доля женщин, не проходивших курс подготовки к экзамену, которые не сдали экзамен по математике: 0.21.

**Задание 11. С помощью groupby выполните задания ниже. Также выведите время выполнения каждого из заданий.**
    * Для каждой этнической группы выведите средний балл за экзамен по чтению
    * Для каждого уровня образования выведите минимальный балл за экзамен по письму

In [ ]:
import time

# Измерение времени: сделаем цикл на 100 иттераций, чтобы подсчитать среднее
# время выполнения задания
execution_time_sum = 0
for i in range(100):
  start_time = time.time()
  df.groupby('race/ethnicity')['reading score'].mean()
  # Метод groupby группирует данные по уникальным значениям в столбце 'race/ethnicity',
  # а затем для каждой группы вычисляется средний балл по чтению с помощью метода mean().

  df.groupby('education')['writing score'].min()
  end_time = time.time()
  execution_time_sum += end_time - start_time

# Решение задания
print(df.groupby('race/ethnicity')['reading score'].mean())
print()
print(df.groupby('education')['writing score'].min())
print()
print('Average execution time: ' + str(execution_time_sum / 100))

race/ethnicity
group A    64.674157
group B    67.352632
group C    69.103448
group D    70.030534
group E    73.028571
Name: reading score, dtype: float64

education
associate's degree    35
bachelor's degree     38
high school           15
master's degree       46
some college          19
some high school      10
Name: writing score, dtype: int64

Average execution time: 0.0012145566940307618


Среднее время выполнения программы без учета времени выполнения print: 0.00122.

**Задание 12. Выполните задание 11 с помощью циклов. Сравните время выполнения.**

In [ ]:
# Выполнение задания
ethnic_groups = df['race/ethnicity'].unique()
for group in ethnic_groups:
  print(group + ' ' +
        str(df[df['race/ethnicity'] == group]['reading score'].mean()))

print()

education_levels = df['education'].unique()
for level in education_levels:
  print(level + ' ' +
        str(df[df['education'] == level]['writing score'].min()))

print()

# Измерение времени
execution_time_sum = 0
for i in range(100):
  start_time = time.time()

  ethnic_groups = df['race/ethnicity'].unique()
  for group in ethnic_groups:
    df[df['race/ethnicity'] == group]['reading score'].mean()

  education_levels = df['education'].unique()
  for level in education_levels:
    df[df['education'] == level]['writing score'].min()

  end_time = time.time()
  execution_time_sum += end_time - start_time

print('Average execution time: ' + str(execution_time_sum / 100))

group B 67.35263157894737
group C 69.10344827586206
group A 64.67415730337079
group D 70.03053435114504
group E 73.02857142857142

bachelor's degree 38
some college 19
master's degree 46
associate's degree 35
high school 15
some high school 10

Average execution time: 0.007012734413146973


Среднее время выполнения программы без учета времени выполнения print: 0.006. Работает дольше, чем аналогичное решение с использованием метода groupby.
Стоит отметить, что для того, чтобы выводить группы в порядке возрастания, может понадобиться дополнительное время.

**Задание 13. Выведите на экран средние баллы студентов по каждому предмету в зависимости от пола и уровня образования. То есть должно получиться количество групп, равных 2 * (число уровней образования), и для каждой такой группы выыведите средний балл по каждому из предметов.**

Это можно сделать с помощью сводных таблиц (pivot_table):

https://www.kaggle.com/kamilpolak/tutorial-how-to-use-pivot-table-in-pandas

In [ ]:
pd.pivot_table(df,index=['gender','education'],
               values=['math score', 'reading score', 'writing score'])
# Можно не указывать values. Тогда выведется такая же таблица, но ее сопроводит
# warning о том, что метод не может посчитать среднее значение
# в столбцах, где значение -- строковые типы данных

math score  reading score  writing score
gender education                                                   
female associate's degree   65.250000      74.120690      74.000000
       bachelor's degree    68.349206      77.285714      78.380952
       high school          59.351064      68.202128      66.691489
       master's degree      66.500000      76.805556      77.638889
       some college         65.406780      73.550847      74.050847
       some high school     59.296703      69.109890      68.285714
male   associate's degree   70.764151      67.433962      65.405660
       bachelor's degree    70.581818      68.090909      67.654545
       high school          64.705882      61.480392      58.539216
       master's degree      74.826087      73.130435      72.608696
       some college         69.009259      64.990741      63.148148
       some high school     67.840909      64.693182      61.375000

#### Сколько студентов успешно сдали экзамен по математике?

Создадим новый столбец в таблице df под названием Math_PassStatus и запишем в него F, если студент не сдал экзамен по математике (балл за экзамен < passmark), и P иначе.

Посчитаем количество студентов, сдавших и не сдавших экзамен по математике.

Сделаем аналогичные шаги для экзаменов по чтению и письму.

In [ ]:
df['Math_PassStatus'] = df['math score'].apply(lambda x: 'F' if x < passmark else 'P')
print(df['Math_PassStatus'].value_counts())
print()

df['Reading_PassStatus'] = df['reading score'].apply(lambda x: 'F' if x < passmark else 'P')
print(df['Reading_PassStatus'].value_counts())
print()

df['Writing_PassStatus'] = df['writing score'].apply(lambda x: 'F' if x < passmark else 'P')
print(df['Writing_PassStatus'].value_counts())
print()

P    865
F    135
Name: Math_PassStatus, dtype: int64

P    910
F     90
Name: Reading_PassStatus, dtype: int64

P    886
F    114
Name: Writing_PassStatus, dtype: int64



Метод apply позволяет применить функцию к каждому элементу в столбце DataFrame. В данном случае, функция — это лямбда-выражение, которое проверяет, меньше ли значение в столбце math score заданного порога passmark. Если условие истинно, в новый столбец Math_PassStatus записывается 'F' (не сдал), иначе — 'P' (сдал).

#### Сколько студентов успешно сдали все экзамены?

Создадим столбец OverAll_PassStatus и запишем в него для каждого студента 'F', если студент не сдал хотя бы один из трех экзаменов, а иначе 'P'.

Посчитаем количество студентов, которые сдали все экзамены.

In [ ]:
df['OverAll_PassStatus'] = df.apply(lambda row: 'F' if row['math score'] < passmark or
                                    row['reading score'] < passmark or
                                    row['writing score'] < passmark else 'P', axis = 1)
# Лямбда-функция принимает в качестве аргумента целую строку row.
# Далее мы обращаемся к нужным столбцам через эту строку.

print(df['OverAll_PassStatus'].value_counts()['P'])

812


Количество студентов, которые сдали все экзамены: 812.

#### Переведем баллы в оценки

### Система перевода баллов в оценки
####    больше 90 = A
####      80-90 = B
####      70-80 = C
####      60-70 = D
####      50-60 = E
####    меньше 50 = F (Fail)

Создадим вспомогательную функцию, которая будет по среднему баллу за три экзамена выставлять оценку студенту по данным выше критериям.

Создадим столбец Grade и запишем в него оценку каждого студента.

Выведим количество студентов, получивших каждую из оценок.

In [ ]:
def GetGrade(average_mark):
    return ('A' if average_mark > 90 else 'B' if average_mark >= 80
            else 'C' if average_mark >= 70 else 'D' if average_mark >= 60
            else 'E' if average_mark >= 50 else 'F')

average_score = df[['math score', 'reading score', 'writing score']].mean(axis=1)
# Эта строка кода сначала выбирает оценки по математике, чтению и письму,
# затем считает средний балл для каждого студента и записывает в average_score.

df['Grade'] = average_score.apply(GetGrade)
df['Grade'].value_counts()

C    261
D    256
E    182
B    148
F    103
A     50
Name: Grade, dtype: int64